# **LangChain Decoded**

## Getting Started

In [ ]:
# Install the LangChain package
!pip install langchain

In [ ]:
# Install the OpenAI package
!pip install openai

In [4]:
# Configure the API key
import os

openai_api_key = os.environ.get('OPENAI_API_KEY', 'sk-XXX')

## Part 3: Prompts

In [ ]:
# Ask the LLM about a recent event/occurence
from langchain.llms.openai import OpenAI

llm = OpenAI(model_name='text-davinci-003', openai_api_key=openai_api_key)

print(llm("What is LangChain useful for? Answer in one sentence."))

In [ ]:
# Ask the same question again, but with relevant context
prompt = """You are a helpful assistant, who can explain concepts in an easy-to-understand manner. Answer the following question succintly.
          Context: There are six main areas that LangChain is designed to help with. These are, in increasing order of complexity:
            LLMs and Prompts: This includes prompt management, prompt optimization, a generic interface for all LLMs, and common utilities for working with LLMs.
            Chains: Chains go beyond a single LLM call and involve sequences of calls (whether to an LLM or a different utility). LangChain provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications.
            Data Augmented Generation: Data Augmented Generation involves specific types of chains that first interact with an external data source to fetch data for use in the generation step. Examples include summarization of long pieces of text and question/answering over specific data sources.
            Agents: Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done. LangChain provides a standard interface for agents, a selection of agents to choose from, and examples of end-to-end agents.
            Memory: Memory refers to persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.
            Evaluation: Generative models are notoriously hard to evaluate with traditional metrics. One new way of evaluating them is using language models themselves to do the evaluation. LangChain provides some prompts/chains for assisting in this.
          Question: What is LangChain useful for?
          Answer: """

print(llm(prompt))

In [ ]:
# Use a template to structure the prompt
from langchain import PromptTemplate

template = """You are a helpful assistant, who is good at general knowledge trivia. Answer the following question succintly.
              Question: {question}
              Answer:"""

prompt = PromptTemplate(template=template, input_variables=['question'])

question = "Who won the first football World Cup?"

print(llm(question))

In [ ]:
# Use a chain to execute the prompt
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.run(question))

In [ ]:
# Save prompt template to JSON file
prompt.save("myprompt.json")

# Load prompt template from JSON file
from langchain.prompts import load_prompt

saved_prompt = load_prompt("myprompt.json")
assert prompt == saved_prompt

print(llm(question))

In [ ]:
# Guide the model using few shot examples in the prompt
from langchain import PromptTemplate, FewShotPromptTemplate

examples = [
    { "question": "How can we extend our lifespan?", 
      "answer": "Just freeze yourself and wait for technology to catch up."},
    { "question": "Does red wine help you live longer?", 
      "answer": "I don't know about that, but it does make the time pass more quickly."},
    { "question": "How can we slow down the aging process?", 
      "answer": "Simple, just stop having birthdays."}
]

template = """
    Question: {question}
    Answer: {answer}
  """

prompt = PromptTemplate(input_variables=["question", "answer"], template=template)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="Respond with a funny and witty remark.",
    suffix="Question: {question}\nAnswer:",
    input_variables=["question"],
    example_separator=""
)

print(few_shot_prompt.format(question="How can I eat healthy?"))
print(llm(few_shot_prompt.format(question="How can I eat healthy?")))

In [ ]:
# Use prompt templates with chat models
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

system_message="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

human_message="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

messages = chat_prompt.format_prompt(input_language="English", output_language="Spanish", text="I'm hungry, give me food.").to_messages()

chat(messages)